<a href="https://colab.research.google.com/github/sergekamanzi/Chat-Bot-/blob/main/chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
import numpy as np
import faiss
from sentence_transformers import SentenceTransformer
import gradio as gr

# Define the question-answer data
data = {
    "What was the significance of 1957 in Rwanda?": "In 1957, the Hutu Manifesto was published by Hutu intellectuals demanding political reform and majority rule, marking a turning point in ethnic tensions.",
    "What happened during the Rwandan Revolution of 1959?": "The Rwandan Revolution in 1959 led to the end of the Tutsi monarchy and the rise of a Hutu-led republic. It triggered ethnic violence and the exile of many Tutsis.",
    "When did Rwanda gain independence?": "Rwanda gained independence from Belgium on July 1, 1962, with Grégoire Kayibanda becoming the first president.",
    "Who was Grégoire Kayibanda?": "Grégoire Kayibanda was Rwanda's first president from 1962 to 1973. He led the country after independence and promoted Hutu nationalism.",
    "What happened in 1973 in Rwanda?": "In 1973, Major General Juvénal Habyarimana overthrew Grégoire Kayibanda in a military coup and established a one-party state under the MRND party.",
    "Who was Juvénal Habyarimana?": "Juvénal Habyarimana was Rwanda's second president, ruling from 1973 until his assassination in 1994. His regime was marked by ethnic favoritism and authoritarianism.",
    "What caused the 1994 Genocide?": "The 1994 Genocide against the Tutsi was caused by long-standing ethnic tensions, political instability, and extremist propaganda. The assassination of President Habyarimana triggered the mass killings.",
    "What happened during the 1994 Genocide in Rwanda?": "From April to July 1994, approximately 800,000 Tutsi and moderate Hutu were killed in a genocide perpetrated by Hutu extremists.",
    "Who ended the 1994 Genocide in Rwanda?": "The Rwandan Patriotic Front (RPF), led by Paul Kagame, ended the genocide in July 1994 after capturing Kigali and taking control of the country.",
    "What is the Rwandan Patriotic Front (RPF)?": "The RPF is a political and military organization that ended the 1994 Genocide and has led the Rwandan government since then. It was founded by Tutsi exiles in Uganda.",
    "Who is Paul Kagame?": "Paul Kagame is Rwanda's current president and former military leader of the RPF. He played a key role in ending the 1994 Genocide and has been president since 2000.",
    "What happened in Rwanda after the Genocide?": "After the Genocide, Rwanda focused on national unity, justice through Gacaca courts, reconstruction, and economic development under RPF leadership.",
    "What are Gacaca courts in Rwanda?": "Gacaca courts were community-based courts used to try suspects of the 1994 Genocide, aimed at truth-telling, justice, and reconciliation.",
    "When did Paul Kagame become president?": "Paul Kagame officially became president of Rwanda in 2000 after Pasteur Bizimungu resigned.",
    "What has Rwanda achieved since 2000?": "Since 2000, Rwanda has made progress in healthcare, education, digital transformation, gender equality, and economic development.",
    "What is Vision 2020 in Rwanda?": "Vision 2020 was a national development plan launched by the Rwandan government to transform the country into a middle-income economy by the year 2020.",
    "What replaced Vision 2020 in Rwanda?": "Vision 2050 replaced Vision 2020 and aims to make Rwanda an upper-middle-income country by 2035 and a high-income country by 2050.",
    "What is Umuganda in Rwanda?": "Umuganda is a monthly national community service day in Rwanda where citizens clean streets, plant trees, and build public infrastructure.",
    "What is Rwanda known for today?": "Rwanda is known for its clean cities, rapid development, eco-tourism, commitment to reconciliation, and being one of the safest countries in Africa.",
    "What is Kwibuka in Rwanda?": "Kwibuka is an annual commemoration of the 1994 Genocide against the Tutsi. It means 'to remember' in Kinyarwanda and begins every April 7."
}

# Load the sentence transformer model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Extract questions and answers
questions = list(data.keys())
answers = list(data.values())

# Encode the questions
question_embeddings = model.encode(questions, convert_to_tensor=False)

# Convert embeddings to numpy array
question_embeddings = np.array(question_embeddings).astype('float32')

# Create a FAISS index
dimension = question_embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(question_embeddings)

# Lists for detecting greetings, gratitude, and farewells
GREETINGS = ["hello", "hi", "hey", "greetings", "good morning", "good afternoon", "good evening"]
GRATITUDE = ["thank you", "thanks", "appreciate it", "thank you so much", "thanks a lot"]
FAREWELLS = ["bye", "goodbye", "see you", "farewell", "take care"]

def is_greeting(query):
    query_lower = query.lower().strip()
    return any(greeting in query_lower for greeting in GREETINGS)

def is_gratitude(query):
    query_lower = query.lower().strip()
    return any(gratitude in query_lower for gratitude in GRATITUDE)

def is_farewell(query):
    query_lower = query.lower().strip()
    return any(farewell in query_lower for farewell in FAREWELLS)

def chatbot(query):
    # Check for greetings
    if is_greeting(query):
        return "Hello! I'm here to assist you with questions about Rwandan history. What would you like to learn about today?"

    # Check for gratitude
    if is_gratitude(query):
        return "You're welcome! I'm glad I could help. Would you like to know more about Rwandan history?"

    # Check for farewells
    if is_farewell(query):
        return "Goodbye! Feel free to come back if you have more questions about Rwandan history."

    # Encode the input query
    query_embedding = model.encode([query], convert_to_tensor=False).astype('float32')

    # Search for the closest question
    k = 1  # Number of nearest neighbors
    distances, indices = index.search(query_embedding, k)

    # Get the distance to the closest match
    distance = distances[0][0]

    # Set a threshold for out-of-domain detection
    DISTANCE_THRESHOLD = 1.5

    if distance > DISTANCE_THRESHOLD:
        return "I'm sorry, I don't have information on that topic. I can help with questions about Rwandan history. What would you like to know?"

    # Get the most similar question and its answer
    matched_question = questions[indices[0][0]]
    matched_answer = answers[indices[0][0]]

    # Add a polite tone and follow-up prompt
    return f"Here’s what I found: {matched_answer} Would you like to know more about Rwandan history?"

# Create a Gradio interface
iface = gr.Interface(
    fn=chatbot,
    inputs=gr.Textbox(lines=2, placeholder="Ask a question about Rwandan history..."),
    outputs="text",
    title="Rwandan History Chatbot",
    description="Ask questions about Rwandan history, and the chatbot will respond based on the provided knowledge base."
)

# Test the chatbot with sample queries
test_queries = [
    "Hello",  # Greeting
    "What happened in Rwanda in 1959?",  # In-domain
    "Thank you so much",  # Gratitude
    "What is the capital of France?",  # Out-of-domain
    "Who is the current president of Rwanda?",  # In-domain
    "Thanks a lot",  # Gratitude
    "Goodbye",  # Farewell
    "What are Gacaca courts?",  # In-domain
    "Bye bye",  # Farewell
]

print("Testing the chatbot with sample queries:")
for query in test_queries:
    response = chatbot(query)
    print(f"Query: {query}")
    print(f"Response: {response}\n")

# Launch the Gradio interface
iface.launch()

Testing the chatbot with sample queries:
Query: Hello
Response: Hello! I'm here to assist you with questions about Rwandan history. What would you like to learn about today?

Query: What happened in Rwanda in 1959?
Response: Here’s what I found: The Rwandan Revolution in 1959 led to the end of the Tutsi monarchy and the rise of a Hutu-led republic. It triggered ethnic violence and the exile of many Tutsis. Would you like to know more about Rwandan history?

Query: Thank you so much
Response: You're welcome! I'm glad I could help. Would you like to know more about Rwandan history?

Query: What is the capital of France?
Response: Here’s what I found: Rwanda is known for its clean cities, rapid development, eco-tourism, commitment to reconciliation, and being one of the safest countries in Africa. Would you like to know more about Rwandan history?

Query: Who is the current president of Rwanda?
Response: Here’s what I found: Since 2000, Rwanda has made progress in healthcare, education, d